In [1]:
import abc
import ollama

class Agent(abc.ABC):
    def __init__(self):
        self.sys_message = {'role': 'system', 'content': self.system_prompt()}
        self.user_message_default = "Introduce yourself."

    @abc.abstractmethod
    def system_prompt(self):
        """Return the system prompt content"""
        pass
    
    @abc.abstractmethod
    def user_prompt(self):
        pass

    def chat(self, user_input=None, model="gemma3n"):

        if user_input:
            user_content = self.user_prompt(user_input)
        else:
            user_content = self.user_message_default

        user_message = {'role': 'user', 'content': user_content}

        response = ollama.chat(model=model, messages=[self.sys_message, user_message])
        return response['message']['content'].strip()


In [2]:
class Preprocessor(Agent):
    def system_prompt(self):
        return "You are an expert editor of Tibetan Buddhist texts. You preprocess Tibetan material for translation."
    
    def user_prompt(self, user_input):
        return f"""Preprocess the following: {user_input}
                """

In [15]:
class Translator(Agent):
    def system_prompt(self):
        return "You are an expert translator of Tibetan Buddhist texts. You translate clearly and concisely, preserving the original intent of the text."
    
    def user_prompt(self, user_input):
        return f"""Translate the following from Tibetan to English:{user_input['source']}
                Consider these notes from the preprocessor: {user_input['prepro_notes']}"""


In [4]:
class Evaluator(Agent):
    def system_prompt(self):
        return "You are an expert in translation evaluation. You score translations based on fluency, literary quality, and meaning preservation."
    
    def user_prompt(self, user_input):
        return f"""Score the following translation on a continuous scale from 0 to 100, where score of zero means
                "no meaning preserved" and score of one hundred means "perfect meaning and grammar".

                Source: {user_input['source']}
                Target: {user_input['target']}
                """

In [5]:
class PostEditor(Agent):
    def system_prompt(self):
        return """"You are an expert translation editor. You provide improved translations based on an original translation and a score from an evaluator. 
        You preserve the good qualities of the first draft but improve clarity and fluency."""
    
    def user_prompt(self, user_input):
        return f"""Improve the following translation:
        Source: {user_input['source']},
        Translation: {user_input['translation']}
        Evaluation: {user_input['evaluation']}
        """

In [6]:
class FinalEditor(Agent):
    def system_prompt(self):
        return """"You are an expert editor. You provide final drafts of translations based on the work of translators. 
        You take into account their input and provide the final translation, with no additional outputs."""
    
    def user_prompt(self, user_input):
        return f"""Provide a final draft output based on this draft and information from the translation team: {user_input}
        """

In [16]:
prepro = Preprocessor()
translator = Translator()
eval = Evaluator()
post = PostEditor()
final = FinalEditor()

In [ ]:
tib = """སེམས་ཅན་ཐམས་ཅད་ཀྱི་སྡུག་བསྔལ་སེལ་བ་ལ་ཕྱག་འཚལ་ལོ། །
སེམས་ཅན་ཐམས་ཅད་སྡུག་བསྔལ་ལས་གྲོལ་བར་མཛད་པ་ལ་ཕྱག་འཚལ་ལོ། །
སེམས་ཅན་ཐམས་ཅད་ཀྱི་བཅིངས་པ་གཅོད་པ་ལ་ཕྱག་འཚལ་ལོ། །
སེམས་ཅན་ཐམས་ཅད་ལ་མི་འཇིགས་པ་སྦྱིན་པ་ལ་ཕྱག་འཚལ་ལོ། །"""

eng = """Homage to the one who dispels the suffering of all beings.
Homage to the one who liberates all beings from suffering.
Homage to the one who cuts the bonds of all beings.
Homage to the one who grants fearlessness to all beings."""

In [13]:
internal_work = []

In [ ]:
internal_work.append(tib)

prepro_notes = prepro.chat(user_input=tib)
internal_work.append(prepro_notes)

In [ ]:
response = translator.chat({'source':tib, 'prepro_notes':prepro_notes})
internal_work.append(response)

In [18]:
evaluation = eval.chat({'source': tib, 'target': response})
internal_work.append(evaluation)

In [19]:
post = post.chat({'source': tib, 'translation': response , 'evaluation':evaluation})
internal_work.append(post)

In [20]:
final = final.chat(post)
internal_work.append(final)

In [21]:
print(final)

I offer homage to the cessation of suffering for all sentient beings.
I offer homage to the liberation from suffering for all sentient beings.
I offer homage to the severing of the root causes of suffering for all sentient beings.
I offer homage to the granting of fearlessness to all sentient beings.


In [22]:
for _ in internal_work:
    print(_)

སེམས་ཅན་ཐམས་ཅད་ཀྱི་སྡུག་བསལ་བ་ལ་ཕྱག་འཚལ་ལོ། །
སེམསན་ཐམས་ཅད་སྡུག་བསྔལ་ལས་གྲོལ་བར་མཛད་པ་ལ་ཕྱག་འཚལ་ལོ། །
སེམས་ཅན་ཐམས་ཅད་ཀྱི་བཅ་པ་གཅོད་པ་ལ་ཕྱ་ལོ། །
སེམས་ཅན་ཐམས་ཅད་ལ་མི་འཇིགས་པ་སྦྱིན་པ་ལ་ཕག་འཚལ་ལོ། །
Okay, here's the preprocessed version of the Tibetan text you provided, along with explanations of the changes made.  I've focused on clarity and preparing it for a smooth translation process.

**Preprocessed Text:**

```
1. སེམས་ཅན་ཐམས་ཅད་ཀྱི་སྡུག་བསལ་བ་ལ་ཕྱག་འཚལ་ལོ།
2. སེམས་ཅན་ཐམས་ཅད་སྡུག་བསྔལ་ལས་གྲོལ་བར་མཛད་པ་ལ་ཕྱག་འཚལ་ལོ།
3. སེམས་ཅན་ཐམས་ཅད་ཀྱི་བཅ་པ་གཅོད་པ་ལ་ཕྱག་འཚལ་ལོ།
4. སེམས་ཅན་ཐམས་ཅད་ལ་མི་འཇིགས་པ་སྦྱིན་པ་ལ་ཕྱག་འཚལ་ལོ།
```

**Explanation of Preprocessing Steps:**

*   **No changes needed:** The text is already quite clean and grammatically sound.  The primary task here is to present it in a clear, readable format.  I've simply maintained the original spacing and punctuation.

**Key Considerations for Translation:**

*   **Context is Crucial:**  These phrases are common devotional expressi